In [12]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-lite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow-lite (from versions: none)
ERROR: No matching distribution found for tensorflow-lite


In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import models,layers
import matplotlib.pyplot as plt

In [14]:
df = pd.read_csv('/content/completeBhageeraPayload.csv')

In [15]:
df.head(5)

,mlid,ax,ay,az,gx,gy,gz
0,Walking,-0.430176,0.611572,-0.747803,-17.633587,-55.404579,4.854962
1,Walking,-0.600586,0.668945,-0.920898,10.916031,88.320610,-46.167938
2,Walking,-0.390137,0.568115,-0.972168,-23.702290,9.648855,-15.167939
3,Walking,-1.108887,0.886475,-0.582275,70.381676,-85.572517,47.000000
4,Walking,-0.442139,0.665039,-0.820068,44.259541,-87.793892,49.152672


In [16]:
df['mlid'].value_counts() #data is imbalanced

Rest        8141
Sleep       5753
Walking     5241
Running     3698
Standing    1757
Sitting      839
Name: mlid, dtype: int64

In [17]:
from sklearn.preprocessing import LabelEncoder
yle = LabelEncoder()
df['mlid'] = yle.fit_transform(df['mlid'])

In [18]:
df['mlid'].value_counts() 

0    8141
3    5753
5    5241
1    3698
4    1757
2     839
Name: mlid, dtype: int64

In [19]:
X = df.drop('mlid', axis =1)
Y= df['mlid']

In [20]:
X.head(5)

,ax,ay,az,gx,gy,gz
0,-0.430176,0.611572,-0.747803,-17.633587,-55.404579,4.854962
1,-0.600586,0.668945,-0.920898,10.916031,88.320610,-46.167938
2,-0.390137,0.568115,-0.972168,-23.702290,9.648855,-15.167939
3,-1.108887,0.886475,-0.582275,70.381676,-85.572517,47.000000
4,-0.442139,0.665039,-0.820068,44.259541,-87.793892,49.152672


In [21]:
from imblearn.combine import SMOTETomek
smk = SMOTETomek(random_state = 42)

In [22]:
# resampling th data
X_resampled , Y_resampled = smk.fit_resample(X, Y)

In [23]:
Y_resampled.value_counts() #data is now balanced

3    7935
2    7850
4    7830
1    7696
5    7483
0    7340
Name: mlid, dtype: int64

In [24]:
# Splitting the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_resampled,Y_resampled, test_size=0.2, random_state=0)

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [26]:
# Convert the labels to one-hot encoded vectors
num_classes = 6 #number of unique classes
y_train_encoded = to_categorical(Y_train, num_classes = num_classes)
y_test_encoded = to_categorical(Y_test, num_classes = num_classes)

In [27]:
y_train_encoded

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [28]:
# Define the architecture of the ANN model
model = Sequential()
model.add(Dense(64, activation = 'relu')) # input_dim represents the number of input features
model.add(Dense(64, activation = 'relu'))
model.add(Dense(num_classes, activation = 'softmax'))   # Output layer with softmax activation for multiclass classification

In [29]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [30]:
model_history = model.fit(X_train, y_train_encoded, epochs = 100, batch_size = 32)

Epoch 1/100
1154/1154 [==============================] - 10s 4ms/step - loss: 1.2190 - accuracy: 0.5705
Epoch 2/100
1154/1154 [==============================] - 8s 7ms/step - loss: 0.7615 - accuracy: 0.7188
Epoch 3/100
1154/1154 [==============================] - 8s 7ms/step - loss: 0.6382 - accuracy: 0.7607
Epoch 4/100
1154/1154 [==============================] - 8s 7ms/step - loss: 0.5804 - accuracy: 0.7825
Epoch 5/100
1154/1154 [==============================] - 4s 3ms/step - loss: 0.5513 - accuracy: 0.7927
Epoch 6/100
1154/1154 [==============================] - 4s 4ms/step - loss: 0.5297 - accuracy: 0.7994
Epoch 7/100
1154/1154 [==============================] - 4s 4ms/step - loss: 0.5129 - accuracy: 0.8064
Epoch 8/100
1154/1154 [==============================] - 4s 3ms/step - loss: 0.4962 - accuracy: 0.8140
Epoch 9/100
1154/1154 [==============================] - 4s 3ms/step - loss: 0.4831 - accuracy: 0.8199
Epoch 10/100
1154/1154 [==============================] - 4s 4ms/step - 

In [31]:
print(model_history.history.keys())

dict_keys(['loss', 'accuracy'])


In [32]:
# Evaluate the model
loss , accuracy = model.evaluate(X_test, y_test_encoded)

289/289 [==============================] - 1s 2ms/step - loss: 0.3685 - accuracy: 0.8684


In [33]:
model.save('/content/my_model')

In [34]:
!ls

completeBhageeraPayload.csv  my_model  sample_data


In [35]:
converter = tf.lite.TFLiteConverter.from_saved_model('/content/my_model')

In [36]:
converter.target_spec.supported_types = [tf.float32]

In [37]:
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

In [38]:
tflite_model = converter.convert()

In [44]:
with open('converted_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [45]:
open("converted_model.tflite", "wb").write(tflite_model)

22384

In [46]:
# Load the converted TFLite model
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
try:
    interpreter.allocate_tensors()
    print("Model converted to TFLite successfully!")
except ValueError as e:
    print("Error loading the TFLite model:", e)

Model converted to TFLite successfully!


In [47]:
!ls

completeBhageeraPayload.csv  converted_model.tflite  my_model  sample_data
